In [6]:
# install dependencies
import sqlite3
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

In [7]:
# connect to database
connection = sqlite3.connect("../../data/colleges.sqlite")
colleges_hd2021 = pd.read_sql_query("SELECT * FROM hd2021_summary", connection)

In [8]:
# separate strings and numbers
is_num = [dtype == np.int64 or dtype == np.float64 for dtype in colleges_hd2021.dtypes]
is_string = [dtype != np.int64 or dtype != np.float64 for dtype in colleges_hd2021.dtypes]

number_colleges_hd2021 = colleges_hd2021.loc[:, is_num]
string_colleges_hd2021 = colleges_hd2021.loc[:, is_string]

In [9]:
# load the model
USE_URL = "https://tfhub.dev/google/universal-sentence-encoder/4"
use = hub.load(USE_URL)
print(f"use {USE_URL} is loaded")

use https://tfhub.dev/google/universal-sentence-encoder/4 is loaded


In [83]:
def find_embeddings_closeness(column):
    unique_items = list(set(column))
    embedded_items = np.array(use(unique_items)).tolist()

    reverse_lookup = dict(zip(unique_items, embedded_items))

    全体のcloseness = {}
    for item, embedding in reverse_lookup.items():
        closeness = {}
        for other_item, other_embedding in reverse_lookup.items():
            # the closeness in relation to the other categories
            closeness[np.inner(embedding, other_embedding)] = other_item

        全体のcloseness[item] = closeness
    
    return 全体のcloseness


def find_closeness(table_columns: dict):
    for table_name, column_names in table_columns.items():
        table_df = pd.read_sql_query(f"SELECT * FROM {table_name}", connection)

        for column_name in column_names:
            column_with_unitid = table_df[["UNITID", column_name]].dropna() # this is for joining the column back into the database
            column_closeness_embeddings = find_embeddings_closeness(list(column_with_unitid[column_name]))

            column_with_unitid.replace({"Largest_Program": column_closeness_embeddings}, inplace=True)

    return column_with_unitid #ONLY WORKS FOR ONE FIX FIX FIX FIX
            
            
            


In [85]:
largest_program_closeness_df = find_closeness({"hd2021_summary": ["Largest_Program"]})
print(largest_program_closeness_df)

      UNITID                                    Largest_Program
0     100654  {0.15073927796282055: 'Building/Property Maint...
1     100663  {0.20456809899343226: 'Building/Property Maint...
2     100690  {0.2406472771372165: 'Building/Property Mainte...
3     100706  {0.20456809899343226: 'Building/Property Maint...
4     100724  {0.045443261254886645: 'Building/Property Main...
...      ...                                                ...
6283  497286  {0.21553619971384613: 'Building/Property Maint...
6284  497301  {0.20092534377193685: 'Building/Property Maint...
6285  497310  {0.20092534377193685: 'Building/Property Maint...
6286  497329  {0.24479444479256723: 'Building/Property Maint...
6287  497338  {0.1863136848206276: 'Building/Property Mainte...

[6089 rows x 2 columns]


In [35]:
# find top 10 for specific embedding
# result = find_column_closeness(largest_programs)
values_for_specific = list(result.values())[2]
top_values = list(values_for_specific.keys())
top_values.sort(reverse=True)
print(top_values[:10])
print([values_for_specific[value] for value in top_values[:10]])

[1.0000001001327767, 0.7190875504116907, 0.7119189097369634, 0.7096008939200131, 0.7039315960573392, 0.6996315170176632, 0.6725764135810686, 0.6475616258345458, 0.6296107073788946, 0.580015005153564]
['Clinical Psychology', 'Clinical, Counseling and Applied Psychology, Other', 'Applied Psychology', 'Counseling Psychology', 'Experimental Psychology', 'Forensic Psychology', 'Psychology, Other', 'Psychology, General', 'Community Psychology', 'Research and Experimental Psychology, Other']
